In [1]:
import torch
import spacy
import tqdm
import numpy as np
import torch.nn as nn
from torchtext.data import Field, BucketIterator, TabularDataset

In [2]:
torch.cuda.get_device_name()

'Tesla T4'

In [3]:
spacy_eng = spacy.load("en")

In [4]:
def tokenizer(text):
    return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

In [5]:
query = Field(sequential=True, tokenize=tokenizer, lower=True, init_token="<sos>", eos_token="<eos>")
ans = Field(sequential=True, tokenize=tokenizer, lower=True, init_token="<sos>", eos_token="<eos>")

fields = {"query": ("q", query), "ans": ("a", ans)}
data, dummy = TabularDataset.splits(path="", train="c-3po-convos.csv", test="c-3po-convos.csv",
                                    format="csv", fields=fields)

In [6]:
len(data)

2141

In [7]:
for d in data:
    print(vars(d)["q"])
    print(vars(d)["a"])
    break

['what', 'is', 'you', 'name', '?']
['hi', ',', 'my', 'name', 'is', 'c-3po']


In [8]:
query.build_vocab(data, max_size=15000, min_freq=2)
ans.build_vocab(data, max_size=15000, min_freq=2)

In [9]:
class Transformer(nn.Module):
    def __init__(self, embedding_size, src_vocab_size, trg_vocab_size, src_pad_idx, num_heads, num_encoder_layers,
                 num_decoder_layers, forward_expansion, p, max_len, device):
        super(Transformer, self).__init__()

        self.src_pad_idx = src_pad_idx
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_positional_embedding = nn.Embedding(max_len, embedding_size)

        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_positional_embedding = nn.Embedding(max_len, embedding_size)

        self.dropout = nn.Dropout(p)
        self.device = device

        self.transformer = nn.Transformer(embedding_size, num_heads, num_encoder_layers, num_decoder_layers, 
                                          forward_expansion, dropout=p)
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
    
    def build_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_len, N = src.shape
        trg_seq_len, N = trg.shape

        src_positions = (torch.arange(0, src_seq_len).unsqueeze(1).expand(src_seq_len, N).to(self.device))
        trg_positions = (torch.arange(0, trg_seq_len).unsqueeze(1).expand(trg_seq_len, N).to(self.device))
        
        src_embedding = self.dropout((self.src_word_embedding(src) + self.src_positional_embedding(src_positions)))
        trg_embedding = self.dropout((self.trg_word_embedding(trg) + self.trg_positional_embedding(trg_positions)))
        
        src_mask = self.build_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_len).to(self.device)

        x = self.transformer(src_embedding, trg_embedding, src_key_padding_mask=src_mask, tgt_mask=trg_mask)
        x = self.fc_out(x)

        return x

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 100
lr = 3e-5
batch_size = 16
src_vocab_size = len(query.vocab)
trg_vocab_size = len(ans.vocab)
embedding_size = 1024
num_heads = 16
num_encoder_layers = 4
num_decoder_layers = 4
p = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = query.vocab.stoi["<pad>"]

In [13]:
device

device(type='cuda')

In [14]:
batches, _ = BucketIterator.splits((data, dummy), batch_size=batch_size, device=device)

In [15]:
del dummy

In [16]:
for b in batches:
    print(b.q.shape)
    print(b.a.shape)
    break

torch.Size([45, 16])
torch.Size([20, 16])


In [17]:
c_3po = Transformer(embedding_size, src_vocab_size, trg_vocab_size, src_pad_idx, num_heads, num_encoder_layers, 
                  num_decoder_layers, forward_expansion, p, max_len, device).to(device)

In [18]:
c_3po

Transformer(
  (src_word_embedding): Embedding(1415, 1024)
  (src_positional_embedding): Embedding(100, 1024)
  (trg_word_embedding): Embedding(1342, 1024)
  (trg_positional_embedding): Embedding(100, 1024)
  (dropout): Dropout(p=0.1, inplace=False)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=1024, out_features=1024, bias=True)
          )
          (linear1): Linear(in_features=1024, out_features=4, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=4, out_features=1024, bias=True)
          (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): Tran

In [19]:
opt = torch.optim.Adam(c_3po.parameters(), lr)
pad_idx = query.vocab.stoi["<pad>"]
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [20]:
def reply(sentence, net=c_3po, query=query, ans=ans, device=device, max_len=50):
    tokens = [tok.text.lower() for tok in spacy_eng.tokenizer(sentence)]
    tokens.insert(0, "<sos>")
    tokens.append("<eos>")

    text2idx = []
    for t in tokens:
        text2idx.append(query.vocab.stoi[t])
    
    text_tensor = torch.LongTensor(text2idx)
    text_tensor = text_tensor.unsqueeze(1).to(device)

    output = [ans.vocab.stoi["<sos>"]]
    for _ in range(max_len):
        trg_tensor = torch.LongTensor(output).unsqueeze(1).to(device)

        with torch.no_grad():
            p = net(text_tensor, trg_tensor)

        pred = p.argmax(2)[-1, :].item()
        output.append(pred)
        if pred == ans.vocab.stoi["<eos>"]:
            break

    output_text = [ans.vocab.itos[idx] for idx in output]
    answer = ""
    for word in output_text:
        answer += word + " "

    return answer

In [21]:
def save_checkpoint(net, opt, filename):
    check_point = {"net_dict": net.state_dict(), "opt_dict": opt.state_dict()}
    torch.save(check_point, filename)
    print("Checkpoint Saved!")

def load_checkpoint(net, opt, filename):
    check_point = torch.load(filename)
    net.load_state_dict(check_point["net_dict"])
    opt.load_state_dict(check_point["opt_dict"])
    print("Checkpoint Loaded!")

In [22]:
inp = "hi, how are you?"

In [23]:
for epoch in range(epochs):
    batch_losses = []
    for batch in tqdm.tqdm(batches, total=len(batches)):
        X = batch.q.to(device)
        y = batch.a.to(device)

        output = c_3po(X, y[:-1, :])
        output = output.reshape(-1, output.shape[2])
        y = y[1:].reshape(-1)

        opt.zero_grad()
        loss = loss_fn(output, y)
        batch_losses.append(loss.item())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(c_3po.parameters(), max_norm=1)
        opt.step()
    
    print(f" Epoch: {epoch} | Loss: {sum(batch_losses)/len(batch_losses):.4f}")
    print(reply(inp))
    print("")

    # if epoch % 10 == 0:
    #     save_checkpoint(c_3po, opt, f"checkpoint-{epoch}.pth.tar")
    #     print("")

100%|██████████| 134/134 [00:11<00:00, 11.21it/s]


 Epoch: 0 | Loss: 5.1821


  1%|          | 1/134 [00:00<00:13,  9.62it/s]

<sos> i 'm !   !   !   !   !   !   !   !   !   !   !   !   !   !   !   !   !   !   !   !   !   !   !   !   



100%|██████████| 134/134 [00:11<00:00, 11.19it/s]


...

 Epoch: 64 | Loss: 0.1015
<sos> i 'm fine !   !   😄 <eos> 



  0%|          | 0/134 [00:00<?, ?it/s]

 Epoch: 65 | Loss: 0.0958
<sos> i 'm fine !   how are you ? <eos> 



 57%|█████▋    | 77/134 [00:07<00:05, 10.42it/s]


KeyboardInterrupt: ignored

In [27]:
def wake_up(net):
    while True:
        inp = input("You: ")
        if inp.lower() == "shut down":
            break

        r = reply(inp, net=net)
        r = str(r).split(" ")[1:-2]
        out = " ".join(r)
        print(f"C-3PO: {out}")

In [32]:
wake_up(c_3po)

You: hi
C-3PO: hi !   !   how are you ?   😄 😄
You:  i am fine and you?
C-3PO: i 'm fine too !   happy to have someone to talk with 😁 . i !   !   😄
You: what is your name?
C-3PO: my name is c-3po , but you can call me 3po
You: ok, my name is harin
C-3PO: nice to meet you !   ☺ ️
You: where are you?
C-3PO: i 'm from argentina , south of the world !   !
You: ohh ok, are you a human?
C-3PO: i 'm not sure , but not sure for humans !
You: do you like dogs?
C-3PO: i love sure !   !   hahaha <unk> to make you like are sure 😄
You: so, you are a robot?
C-3PO: i 'm c-3po , but you can call me 3po 😁
You: what is your favorite programming language?
C-3PO: <unk> !   !   😍 . y ?
You: shut down
